## Mise en production avec Azure ml service
Laurent Cetinsoy

Avoir des modèles, c'est bien, qu'ils soient utilisable en production, c'est mieux. 
Dans ce notebook on va s'amuser (ou galérer, c'est selon votre rapport à la vie) à mettre en production un modèle entraîné. 

La référence ci dessous va pouvoir nous servir: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where


Les classes et notions intéressantes sont : WebService, InferenceConfig, le script de scoring (que j'appellerai plutôt de prédiction)


Autre référence à regarder pour l'executer par batch : 
  - https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-parallel-run-step

In [1]:
#charger ws

## Mise en production et  Web services

Il existe plusieurs manière de mettre en production via un web service. Cela ce traduit par le fait que la classe WebService a plusieurs classes filles. Quelles sont-elles ? 

LocalWebService, AciWebService

## Script de scoring

Le script de "scoring" permet de spécifier comment le azure ml service va utiliser votre modèle pour faire des prédiction chaque fois que le webservice reçoit une requête. 

ref utile : https://docs.microsoft.com/en-us/learn/modules/register-and-deploy-model-with-amls/2-deploy-model (section 2)

Dans ce script il y a deux fonctions à compléter : les deux fonctions init (pour charger votre beau modèle) et run. 


Créer un environnement avec la méthode from_conda_specification et le fichier env.yaml qui avait été fourni

Créer une instance de la classe **InferenceConfig** qui utilise le script fake_predict.py

Avant de déployer le script en local ou sur azure. Assurez vous que vos fonction init et run marchent bien 
en lançant le fichier de prediction avec python directement. Afficher le résultat de la fonction run. Cela doit renvoyer la valeur 42. Est ce que ça marche

Ce magnifique faux modèle renvoie toujours 42. On va le déployer avec un azure machine learning. D'abord en local et ensuite sur un vrai service

## Déploiement local avec LocalWebService


**Si ça marche pas en local** créer un notebok sur Azure ou passer directement à la mise en production "normale" et ne perdez pas trop de temps. 

Il est conseillé de lancer les commandes suivantes dans un fichier python pour avoir les feedback

Pour déployer un WebService, on créer d'abord un objet configuration. 

Importer la classe LocalWebService 

Avec la méthode deploy_configuration, créer un objet configuration

Avec la méthode **deploy** de la classe Model, "deployez le modèle" localement.  Attention ça demande docker, donc pour les personnes sur windows... essayez Windows subsystem linux mais sinon ne perdez pas trop de temps et sauvegarder le resultat dans la variable **service**. 
Vous pouvez mettre une liste vide pour l'arguments "models"

Utiliser la méthode wait_for_deployment sur votre service et voir si ça marche

Afficher l'url du webservice pour envoyer des requêtes pour faire une prédiction à l'aide de l'attribut **scoring_uri** de votre web_service

Faire une requête http avec la librairie request (ou reprendre celle ci) et vérifier que ça renvoie bien **42**: 

ref utile https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-consume-web-service#call-the-service-python



Maintenant que ça marche on va déployer notre magnifique modèle qui retourne toujours 42 sur azure ! 

Afficher les logs de votre service avec la méthode **get_logs** de votre service

Le module logging de python est un module qui permet de collecter et centraliser les logs. 

il est utilisé par azml. 


si vous avez un problème executer la cellule suivante :

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

A quoi sert-elle ? cf https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-enable-logging

## Faux modèle déployé sur azure container instance

Il existe plusieurs manières de déployer un modèle dans un webservice sur azure. 
Dans un premier temps on va utiliser les Azure Container instance. 

En gros, notre notre modèle sera dans un webservice (une sorte de flask), 
lequel sera dans executé dans un conteneur (docker probablement).


Reference utile : https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-azure-container-instance

Importer la classe AciWebService qui représente un web service tournant dans une Azure Container instance

Créer un objet deployment_config avec la méthode AciWebservice.deploy_configuration

Toujours avec le même objet inference config, deployer le modèle dans un AciWebservice avec la méthode **deploy** de la classe **Model** et récupérer le résultat dans la variable *service*

Appeler la méthode wait_for_deployment avec l'argument show_output=True et attendre longtemps

Vérifier sur le portail azure que le déploiement est bien encours en attendant

Une fois que c'est déployé, récupérer l'url où il faut envoyer la requête avec l'attribut scoring_uri d'objet service

Envoyer une requête avec votre service

Activer l'authentification sur votre service pour que votre modèle ne soit pas accèssible pour tout le monde

Ajouter le header authentication à votre requête avec la clé d'authentication pour que les requête marchent avec l'authentication

## Bonus : déploiement d'un vrai modèle sur azure instance

On va, en gros, reprendre les même étapes que précédemment.

Etape 0 (optionel si vous avez déjà un modèle entraîné sur azure). Entraîner un modèle simple et le register sur Azure

Etape 1 : faire une copie du script predict_fake et modifier la copie : 
   - pour qu'il charge un modèle depuis votre workspace dans la fonction init
   - que lla fonction run utilisé le modèle qui a été chargé dans la variable gloabale model pour faire les prediction avec les données "data". 

Etap2 : refaire la procédure précédente pour que le modèle soit déployé